In [3]:
import numpy as np
import cv2
import glob
from skvideo.io import vreader
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import pickle
import os
#Length of the extracted feature
FEATURE_LENGTH = 3
#How many frames to sample from a video
SAMPLE_NUM = 50 
#The method for frame sampling from a video
SAMPLE_METHOD = "random_K"
root = "/home/jeffshih/work/develop/ND_classifier"

In [25]:
def get_video_length(video_path):
    cap = cv2.VideoCapture(video_path)
    i = 0
    while(cap.isOpened()):
        ret, frame = cap.read()
        if frame is None:
            break
        i +=1 
    return i


def get_video_paths(target_dir):
    video_paths = glob.glob(target_dir + "/*.mp4")
    return video_paths

def extract_frames(video_path, samples=3, method="top_K"):
    
    frames = []
    if method == "top_K":
        cap = cv2.VideoCapture(video_path)
        while(cap.isOpened()):
            ret, frame = cap.read()
            frames.append(frame)
            if len(frames) >= samples:
                return frames

            
        
    elif method == "random_K":
        video_length = get_video_length(video_path)
        sample_index = np.random.choice(video_length, samples)

        cap = cv2.VideoCapture(video_path)
        i = 0
        while(cap.isOpened()):
            ret, frame = cap.read()
            if frame is None:
                break
            elif i in sample_index:
                frames.append(frame)

            i +=1 
    cap.release()
    return frames


def extract_feature(img):
    frame = img
    roiWid = 10
    #roiEdg = 8
    roiEdg = 10
    src_height, src_width, src_channels = frame.shape
    roiX = int(src_width / roiWid)
    roiWidth = roiX * roiEdg
    roiY = int(src_height / roiWid)
    roiHeight = roiY * roiEdg
    RoiImg = frame[roiY : roiY+roiHeight, roiX : roiX+roiWidth]
    
    
    HSV = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
    H,S,V = cv2.split(HSV)
    Vue_STD = np.std(V)
    Gimg = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    G_STD = np.std(Gimg)
    hist,bins = np.histogram(img.flatten(),256,[0,256])
    hist_STD = np.std(hist)
    hist_Mean = np.mean(hist)
    r = img[...,2]
    g = img[...,1]
    b = img[...,0]
    rY = 0.212655;
    gY = 0.715158;
    bY = 0.072187;
    inner = RoiImg
    ir = inner[...,2]
    ig = inner[...,1]
    ib = inner[...,0]

    Luminance = r*0.2126+g*0.7152+0.0722*b
    Li = ir*0.2126+ig*0.7152+0.0722*ib
    hei,wid,chan = img.shape
    ih,iw,ic = inner.shape
    Lum_value = (np.sum(Luminance)-np.sum(Li))/((hei*wid*255)-(ih*iw*255))
    
    mr = np.sqrt((np.mean(r)-119)*(np.mean(r)-119))
    mg = np.sqrt((np.mean(g)-119)*(np.mean(g)-119))
    mb = np.sqrt((np.mean(b)-119)*(np.mean(b)-119))
    GW_value = (mr+mg+mb)/(3*255)
    
    return (Vue_STD,Lum_value,G_STD,GW_value)

### This part is for retraining the model

In [26]:
def generate_data(root):
    input_dirs = {"light":"Video_Light", "dark":"Video_Dark"}
    frames = []
    Y = []
    Xl = []
    Xd = []
    os.chdir(root)
    frame_cnt = 0
    for luminance, input_dir in input_dirs.items():
        is_dark = True if luminance == "dark" else False   
        video_paths = get_video_paths(input_dir)
        #print video_paths
        for video_path in video_paths:
            os.chdir(root)
            new_frame = extract_frames(video_path, SAMPLE_NUM, SAMPLE_METHOD)
            Y += ([is_dark] * len(new_frame))
            for frame in new_frame :
                frame = cv2.resize(frame,(150,120))
                if (is_dark):
                    Xd.append(extract_feature(frame))
                else:
                    Xl.append(extract_feature(frame)) 
                frame_cnt +=1
            print "Processing : " + video_path


    Y = np.array(Y, dtype=bool)
    X = Xl + Xd
    return Xl,Xd,X,Y

In [27]:
#Xl,Xd,X,Y = generate_data(root)
Xl_testset, Xd_testset, X_testset, Y_testset = generate_data("/home/jeffshih/work/develop/ND_classifier/Test_data")

['Video_Dark/Webcam_South_pm2.mp4', 'Video_Dark/Central_Memorial_pm.mp4', 'Video_Dark/8_StreetCalgary_pm.mp4', 'Video_Dark/Free_Preview_pm.mp4', 'Video_Dark/Two_Friends_Roof_pm.mp4', 'Video_Dark/LIVE_Berlin_pm2.mp4', "Video_Dark/City_Toomer's_pm_2.mp4", 'Video_Dark/Sweetwater_TN.mp4', "Video_Dark/City_Toomer's_pm_3.mp4", 'Video_Dark/Grand_Avenue_pm.mp4', 'Video_Dark/Last _Purple_pm.mp4', 'Video_Dark/City_Auburn_Ross _pm2.mp4', 'Video_Dark/Webcam_South_pm3.mp4', 'Video_Dark/Free_Preview_pm2.mp4', "Video_Dark/Moe's_BBQ_pm_2.mp4", 'Video_Dark/LIVE_Berlin_pm.mp4', "Video_Dark/Moe's_BBQ_am.mp4", 'Video_Dark/Teton_Village_pm.mp4', 'Video_Dark/Archie_Europe_pm.mp4', 'Video_Dark/Two_Cruise_Ships.mp4', 'Video_Dark/RailCam_Road_pm.mp4', "Video_Dark/City_Toomer's_pm.mp4", 'Video_Dark/City_Auburn_Ross _pm.mp4', 'Video_Dark/Two_Friends_Roof_am.mp4', 'Video_Dark/Varanger_Kirkenes.mp4', "Video_Dark/Moe's_BBQ_pm_3.mp4", 'Video_Dark/Chioggia_Sotto_pm.mp4', 'Video_Dark/LIVE_Berlin_pm3.mp4', "Video_Dark/

In [37]:
cnt = 0
for i in range(0,len(X_testset)):
    Y = SVM_classifier_normalized.predict([X_testset[i]])
    if (Y == Y_testset[i]):
        cnt +=1
        
print cnt,len(Y_testset)
print float(cnt)/len(Y_testset)

1567 3137
0.499521836149


In [6]:
from sklearn import svm
SVM_classifier = svm.SVC(gamma = 0.001)

#Xl,Xd,X,Y = generate_data(root)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=0)
SVM_classifier.fit(X_train,Y_train)
predicted = SVM_classifier.predict(X_test)
cnt = 0
for i in range(0,len(predicted)):
    if (predicted[i] == Y_test[i]):
        cnt +=1
print cnt,len(predicted)
print float(cnt)/len(predicted)

569 762
0.746719160105


### Generating testing set

In [14]:
test_file_list = []
test_label_list = []
for root, subroot, files in os.walk("./Test_data"):
    for file_name in files :
        if file_name.endswith("jpg"):
            test_file_list.append(os.path.join(root,file_name))
            if "Dark" in root :
                test_label_list.append(True)
            else:
                test_label_list.append(False)   

                

In [8]:
features = []
for i in range(0,len(test_label_list)):
    img = test_file_list[i]
    img = cv2.imread(img)
    #feature = [extract_feature(img)]
    #Y = SVM_classifier.predict(feature)
    #Y_.append(Y)
    inp = extract_feature(img)
    features.append(inp)
    #if (Y[0] == test_label_list[i]):
    #    cnt +=1

In [9]:
from sklearn import preprocessing
normalized_feature = preprocessing.normalize(features)
normalized_X = preprocessing.normalize(X_test)

### Loading model

In [35]:
from sklearn.externals import joblib
os.chdir(root)
SVM_classifier = joblib.load('SVM_model.pkl')

### Test on the split from the training data

In [18]:
#Xl,Xd,X,Y = generate_data(root)
X_train, X_test, Y_train, Y_test = train_test_split(normalized_X, Y, test_size=0.1, random_state=1)
#SVM_classifier_normalized = svm.SVC(gamma = 0.001)
#SVM_classifier_normalized.fit(normalized_feature,test_label_list)
predicted = SVM_classifier.predict(X_test)
cnt = 0
for i in range(0,len(predicted)):
    if (predicted[i] == Y_test[i]):
        cnt +=1
print cnt,len(predicted)
print float(cnt)/len(predicted)

ValueError: Found input variables with inconsistent numbers of samples: [762, 1]

### Test on the seperate image set to eliminate the bias

In [44]:
cnt = 0
Y_ = []
features = []

for i in range(0,len(test_label_list)):
    img = test_file_list[i]
    img = cv2.imread(img)
    #feature = [extract_feature(img)]
    #Y = SVM_classifier.predict(feature)
    #Y_.append(Y)
    inp = extract_feature(img)
    features.append(inp)
    if (inp[0] > 0.55):
        Y = True
    if (Y == test_label_list[i]):
        cnt +=1
    #break
        
print float(cnt) / len(test_label_list)
print len(test_label_list)

0.41112701984
4889


In [40]:
SVM_new = svm.SVC(gamma = 0.0001)
SVM_new.fit(features,test_label_list)

predicted = SVM_new.predict(X_test)
cnt = 0
for i in range(0,len(predicted)):
    if (predicted[i] == Y_test[i]):
        cnt +=1
print cnt,len(predicted)
print float(cnt)/len(predicted)

281 762
0.368766404199


### Dumping the model

In [ ]:
from sklearn.externals import joblib
joblib.dump(SVM_classifier,'SVM_model.pkl')

### For plotting, can not execute.

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(121,projection ='3d')
ax.scatter([i[0] for i in Xl],[i[2] for i in Xl],[i[1] for i in Xl],c='r',marker = 'o')
ax.scatter([i[0] for i in Xd],[i[2] for i in Xd],[i[1] for i in Xd],c='b',marker = 'x')
ax.view_init(45, 45)
ax1 = fig.add_subplot(122,projection ='3d')
ax1.scatter([i[0] for i in Xl],[i[2] for i in Xl],[i[1] for i in Xl],c='r',marker = 'o')
ax1.scatter([i[0] for i in Xd],[i[2] for i in Xd],[i[1] for i in Xd],c='b',marker = 'x')
ax1.view_init(0, 113)
plt.show()

In [ ]:
import timeit
src = '/home/jeffshih/work/develop/ND_classifier/Test_data/Dark_image_2/N029.jpg'
img = cv2.imread(src)
%timeit extract_feature(img)
feature = [extract_feature(img)]
%timeit SVM_classifier.predict(feature)